In [ ]:
!pip show pytorch-pretrained-bert

In [ ]:
!pip install seqeval

In [ ]:
!pip install pytorch-pretrained-bert==0.4.0

In [ ]:
!pip install git+https://github.com/IINemo/libact.git@seq

In [ ]:
#!pip install -e ./packages/libact/
!pip uninstall -y actleto
!pip install git+https://github.com/IINemo/active_learning_toolbox.git@seq

# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys

In [2]:
import torch

device = torch.device("cuda")
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

Tesla V100-DGXS-16GB


In [3]:
from logger import initialize_logger
logger = initialize_logger('biomed_ie')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import json

# Global parameters

In [5]:
# CACHE_DIR = '../workdir/models/0.4.0'

# # MAX_LEN = 150
# BATCH_SIZE = 40
# MAX_LEN = 100
# #BATCH_SIZE = 100
# #BATCH_SIZE_PRED = 1200 # 1600
# BATCH_SIZE_PRED = 1300 # 1600
# random_state = 2019

# EARLY_STOPPING = 4
# MAX_RETRAIN_EPOCHS = 17

# N_AL_PASSES = 10
# N_AL_ITERATIONS = 25

# BASE_LR = 5e-5


CACHE_DIR = '../workdir/models/0.4.0'

# MAX_LEN = 150
BATCH_SIZE = 32
MAX_LEN = 100
#BATCH_SIZE = 100
#BATCH_SIZE_PRED = 1200 # 1600
BATCH_SIZE_PRED = 1300 # 1600
random_state = 2019

EARLY_STOPPING = 1
MAX_RETRAIN_EPOCHS = 20

N_SAMPLES_PER_AL_ITER = 30
N_AL_PASSES = 10
N_AL_ITERATIONS = 25

BASE_LR = 5e-5

VALIDATION_RATIO = 0.25

MAX_TO_ANNEAL = 1

OUTPUT_FILE_PATH = '../workdir/experiments/14_biobert/'

# Load dataset

In [6]:
from pytorch_pretrained_bert import BertTokenizer
bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, do_lower_case=False)

In [7]:
from flair.data_fetcher import NLPTaskDataFetcher

data_folder = '../workdir/genia/conll/'
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder, {0 : 'text', 1 : 'ner'},
                                               train_file='Genia4ERtask1.iob2',
                                               test_file='Genia4EReval1.iob2',
                                               dev_file='Genia4EReval1.iob2')

2019-08-20 10:05:04,116 Reading data from ../workdir/genia/conll
2019-08-20 10:05:04,117 Train: ../workdir/genia/conll/Genia4ERtask1.iob2
2019-08-20 10:05:04,118 Dev: ../workdir/genia/conll/Genia4EReval1.iob2
2019-08-20 10:05:04,119 Test: ../workdir/genia/conll/Genia4EReval1.iob2


# Run experiments

In [8]:
from bert_active_learning_exp import run_experiment_al
from active_learning_seq import LibActNN, LibActNNPositiveLessCertain, RandomSamplingWithRetraining
from libact.query_strategies import UncertaintySampling, RandomSampling
from bert_utils import create_model_optimizer


def create_libact_adaptor(tag2index, adaptor_type, *args):
    model_ctor = lambda :  create_model_optimizer(tag2index, 
                                                      cache_dir=CACHE_DIR, 
                                                      base_lr=BASE_LR, 
                                                      bert_model='../workdir/bio_bert/torch2/')

    return adaptor_type(*args, 
                        model_ctor=model_ctor,
                        batch_size=BATCH_SIZE,
                        bs_pred=BATCH_SIZE_PRED,
                        train_from_scratch=True,
                        retrain_epochs=MAX_RETRAIN_EPOCHS,
                        early_stopping=EARLY_STOPPING)

In [9]:
def create_i2b2_experiment_pack(corpus, attr):
     return [
#     {
#         'corpus' : corpus,
#         'bpe_tokenizer' : bpe_tokenizer,
#         'output_file_path' : '../workdir/experiments/11_biobert/',
#         'ranking_strategy' : lambda trn_ds, _libact_model: UncertaintySampling(trn_ds, model=_libact_model, method='lc'),
#         'libact_adaptor_ctor' : lambda tag2index: create_libact_adaptor(tag2index, LibActNN),
#         'name' : f'{attr}_MNLP',
#         'n_passes' : N_AL_PASSES,
#         'n_al_iterations' : N_AL_ITERATIONS,
#         'max_len' : MAX_LEN,
#         'max_samples_number' : N_SAMPLES_PER_AL_ITER
#     },
    {
        'corpus' : corpus,
        'bpe_tokenizer' : bpe_tokenizer,
        'output_file_path' : '../workdir/experiments/14_biobert/',
        'ranking_strategy' : lambda trn_ds, _libact_model: UncertaintySampling(trn_ds, model=_libact_model, method='lc'),
        'libact_adaptor_ctor' : lambda tag2index: create_libact_adaptor(tag2index, 
                                                                        LibActNNPositiveLessCertain, 
                                                                        [e for e in tag2index.keys() 
                                                                         if e.startswith('B')]),
        'name' : f'{attr}_MNLP-mod',
        'n_passes' : N_AL_PASSES,
        'n_al_iterations' : N_AL_ITERATIONS,
        'max_len' : MAX_LEN,
        'max_samples_number' : N_SAMPLES_PER_AL_ITER
    },
    {
        'corpus' : corpus,
        'bpe_tokenizer' : bpe_tokenizer,
        'output_file_path' : '../workdir/experiments/14_biobert/',
        'ranking_strategy' : lambda trn_ds, _libact_model: RandomSamplingWithRetraining(trn_ds, model=_libact_model, method='lc'),
        'libact_adaptor_ctor' : lambda tag2index: create_libact_adaptor(tag2index, LibActNN),
        'name' : f'{attr}_i.i.d.',
        'n_passes' : N_AL_PASSES,
        'n_al_iterations' : N_AL_ITERATIONS,
        'max_len' : MAX_LEN,
        'max_samples_number' : N_SAMPLES_PER_AL_ITER
    }
]

In [10]:
experiments = create_i2b2_experiment_pack(corpus, 'genia')

In [ ]:
for exp in experiments:
    logger.info('#################### Experiment ######################')
    logger.info(exp)
    
    run_experiment_al(**exp)
    
    logger.info('################### Experiment finished ##############')

2019-08-20 10:06:22,674 - biomed_ie - INFO - #################### Experiment ######################
2019-08-20 10:06:22,675 - biomed_ie - INFO - {'corpus': <flair.data.TaggedCorpus object at 0x7f6a63e55da0>, 'bpe_tokenizer': <pytorch_pretrained_bert.tokenization.BertTokenizer object at 0x7f6b030846a0>, 'output_file_path': '../workdir/experiments/14_biobert/', 'ranking_strategy': <function create_i2b2_experiment_pack.<locals>.<lambda> at 0x7f6a64165b70>, 'libact_adaptor_ctor': <function create_i2b2_experiment_pack.<locals>.<lambda> at 0x7f69d31e0c80>, 'name': 'genia_MNLP-mod', 'n_passes': 10, 'n_al_iterations': 25, 'max_len': 100, 'max_samples_number': 30}
2019-08-20 10:06:22,677 - biomed_ie - INFO - Preparing data...
2019-08-20 10:06:45,250 - biomed_ie - INFO - Done.
2019-08-20 10:06:45,251 - biomed_ie - INFO - Active learning...
2019-08-20 10:06:45,252 - biomed_ie - INFO - =============== Pass: 0 =============
2019-08-20 10:06:45,322 - biomed_ie - INFO - Number of seeding examples: 14

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-08-20 10:06:58,886 - biomed_ie - INFO - Train loss: 0.002055953576908828
2019-08-20 10:06:58,962 - biomed_ie - INFO - Validation loss: 0.0010234704877334373


Epoch:   5%|▌         | 1/20 [00:02<00:41,  2.17s/it]

2019-08-20 10:06:59,646 - biomed_ie - INFO - Train loss: 0.0014401814347436185
2019-08-20 10:06:59,720 - biomed_ie - INFO - Validation loss: 0.0007254080169011337


...

2019-08-20 10:12:01,505 - biomed_ie - INFO - Train loss: 6.284219773665774e-05
2019-08-20 10:12:01,631 - biomed_ie - INFO - Validation loss: 5.330069696407983e-05


Epoch:  65%|██████▌   | 13/20 [00:21<00:11,  1.66s/it]

2019-08-20 10:12:03,193 - biomed_ie - INFO - Train loss: 5.342699027130344e-05
2019-08-20 10:12:03,320 - biomed_ie - INFO - Validation loss: 5.6756303133306214e-05
2019-08-20 10:12:03,321 - biomed_ie - INFO - Bad epoch #1


Epoch:  70%|███████   | 14/20 [00:23<00:09,  1.66s/it]

2019-08-20 10:12:04,875 - biomed_ie - INFO - Train loss: 4.6008487627836355e-05
2019-08-20 10:12:05,002 - biomed_ie - INFO - Validation loss: 5.572660975830424e-05
2019-08-20 10:12:05,003 - biomed_ie - INFO - Bad epoch #2


# Analyze results

In [17]:
from exp_utils import print_al_stats

In [18]:
data_path = '../workdir/experiments_draw/diabetes_MNLP-mod.npy'
entire_dat_score = 0.718
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.6513872668796481
Entire data score: 0.718
Score ratio: 0.9072246056819612


In [19]:
data_path = '../workdir/experiments_draw/cad_MNLP-mod.npy'
entire_dat_score = 0.401
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.2841254596866301
Entire data score: 0.401
Score ratio: 0.708542293482868


In [20]:
data_path = '../workdir/experiments_draw/hypertension_MNLP-mod.npy'
entire_dat_score = 0.739
print_al_stats(data_path, entire_dat_score)

(10,)
Al score:  0.6053099221998488
Entire data score: 0.739
Score ratio: 0.8190932641405261


In [ ]:
import numpy as np

data_path = '../workdir/experiments_draw/diabetes_MNLP-mod.npy'
stats = np.load(data_path)

stats.mean()